In [0]:
# Import functions
from pyspark.sql.functions import col, current_timestamp

# Define variables used in code below
file_path = "/mnt/green-data/Bronze"
username = spark.sql("SELECT regexp_replace(current_user(), '[^a-zA-Z0-9]', '_')").first()[0]
table_name = "genmix_silver"
checkpoint_path = f"/tmp/{username}/_checkpoint/etl_quickstart_2"

# Clear out data from previous demo execution
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
dbutils.fs.rm(checkpoint_path, True)

# Configure Auto Loader to ingest JSON data to a Delta table
(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .option("cloudFiles.includeExistingFiles",False)
  .load(file_path)
  .select("*", col("_metadata.file_path").alias("source_file"), current_timestamp().alias("processing_time"))
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .option("cloudFiles.inferColumnTypes", "true")
  .trigger(availableNow=True)
  .toTable(table_name))


In [0]:
df = spark.read.table(table_name)

In [0]:
display(df)

from,generationmix,to,_rescued_data,source_file,processing_time


In [0]:
%sql
create table genmix_gold as
select
  cast(trim(TRAILING 'Z' FROM `from`) as timestamp) ts,
  cast(generationmix:[0].perc as double)  biomass,
  cast(generationmix:[1].perc as double)  coal,
  cast(generationmix:[2].perc as double)  imports,
  cast(generationmix:[3].perc as double)  gas,
  cast(generationmix:[4].perc as double)  nuclear,
  cast(generationmix:[5].perc as double)  other,
  cast(generationmix:[6].perc as double)  hydro,
  cast(generationmix:[7].perc as double)  solar,
  cast(generationmix:[8].perc as double) wind
from genmix_silver


num_affected_rows,num_inserted_rows


In [0]:
%sql


SELECT max(processing_time) from genmix_silver

max(processing_time)
null
